In [7]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns

## **Read in CSV Files**:

In [8]:
train_values_df = pd.read_csv('data/training_set_values.csv')
train_values_df.head(2) 

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,...,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
0,69572,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0,...,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
1,8776,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,...,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe


In [9]:
test_values_df = pd.read_csv('data/test_set_values.csv')
test_values_df.head(2)

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,...,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
0,50785,0.0,2013-02-04,Dmdd,1996,DMDD,35.290799,-4.059696,Dinamu Secondary School,0,...,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,other,other
1,51630,0.0,2013-02-04,Government Of Tanzania,1569,DWE,36.656709,-3.309214,Kimnyak,0,...,never pay,soft,good,insufficient,insufficient,spring,spring,groundwater,communal standpipe,communal standpipe


In [10]:
submission_format_df = pd.read_csv('data/submission_format.csv')
submission_format_df.head()

,id,status_group
0,50785,predicted label
1,51630,predicted label
2,17168,predicted label
3,45559,predicted label
4,49871,predicted label


In [12]:
train_labels_df = pd.read_csv('data/training_set_labels.csv')


## **Merging Dataframes Above**

In [13]:
# merging train_values_df & train_labels_df
train_values_df = pd.merge(train_values_df, train_labels_df, on='id', how='left')
# merging test_values_df & submission_format_df
test_values_df = pd.merge(test_values_df, submission_format_df, on='id', how='left')

---

## **EDA**

In [14]:
print(f'train_values_df columns: \n{list(train_values_df.columns)}\n{train_values_df.shape}\n')
print(f'test_values_df columns: \n{list(test_values_df.columns)}\n{test_values_df.shape}\n')
print(f'submission_format_df columns: \n{list(submission_format_df.columns)}\n{submission_format_df.shape}\n')
print(f'train_labels_df columns: \n{list(train_labels_df.columns)}\n{train_labels_df.shape}\n')

train_values_df columns: 
['id', 'amount_tsh', 'date_recorded', 'funder', 'gps_height', 'installer', 'longitude', 'latitude', 'wpt_name', 'num_private', 'basin', 'subvillage', 'region', 'region_code', 'district_code', 'lga', 'ward', 'population', 'public_meeting', 'recorded_by', 'scheme_management', 'scheme_name', 'permit', 'construction_year', 'extraction_type', 'extraction_type_group', 'extraction_type_class', 'management', 'management_group', 'payment', 'payment_type', 'water_quality', 'quality_group', 'quantity', 'quantity_group', 'source', 'source_type', 'source_class', 'waterpoint_type', 'waterpoint_type_group', 'status_group']
(59400, 41)

test_values_df columns: 
['id', 'amount_tsh', 'date_recorded', 'funder', 'gps_height', 'installer', 'longitude', 'latitude', 'wpt_name', 'num_private', 'basin', 'subvillage', 'region', 'region_code', 'district_code', 'lga', 'ward', 'population', 'public_meeting', 'recorded_by', 'scheme_management', 'scheme_name', 'permit', 'construction_year',

---

## **Identify Missing Values**

#### (7) columns with missing values in **train_values_df**:

In [15]:
train_values_df.isna().sum().sort_values(ascending=False).head(7)

scheme_name          28166
scheme_management     3877
installer             3655
funder                3635
public_meeting        3334
permit                3056
subvillage             371
dtype: int64

#### (7) columns with missing values in **test_values_df**:

In [16]:
test_values_df.isna().sum().sort_values(ascending=False).head(7)

scheme_name          7092
scheme_management     969
installer             877
funder                869
public_meeting        821
permit                737
subvillage             99
dtype: int64

#### No values missing in **submission_format_df**:

In [17]:
submission_format_df.isna().sum().sort_values(ascending=False)

id              0
status_group    0
dtype: int64

#### No values missing in **train_labels_df**:

In [18]:
train_labels_df.isna().sum().sort_values(ascending=False)

id              0
status_group    0
dtype: int64

---

## **Data Limitations & Cleaning**

What we do to **train_values_df** we will do to **test_values_df**:
1. **Dropping Columns**:
- For example, **'scheme_name'** is missing 28,166 values out of 59,400 in train_values_df. As a result, we will drop this column and others we wont need.
2. **Dropping rows with missing values**:
- For rows with a few missing values, we will drop the rows so we can preserve the columns.
3. **Data Type Conversion**:
- The 'date_recorded' column was the only column that needed to be changes to datetime
4. **Renaming Columns**:
- Many of the columns have names that are confusing or dont represent the data. These have been changed.

In [19]:
# Dropping unneeded columns
dropped_columns = ['scheme_name', 'num_private', 'region_code', 'district_code',
                   'public_meeting', 'recorded_by', 'extraction_type',
                   'extraction_type_group', 'management', 'payment_type',
                   'water_quality', 'quantity_group', 'source_type', 'waterpoint_type'
                   ]
train_values_df = train_values_df.drop(columns=dropped_columns)
test_values_df = test_values_df.drop(columns=dropped_columns)


# Dropping rows with missing values
dropped_rows = ['scheme_management', 'installer','funder', 
                'permit', 'subvillage'
                ]
train_values_df.dropna(subset=dropped_rows, inplace=True)
test_values_df.dropna(subset=dropped_rows, inplace=True)


# Data Type Conversion
train_values_df['date_recorded'] = pd.to_datetime(train_values_df['date_recorded'])
test_values_df['date_recorded'] = pd.to_datetime(test_values_df['date_recorded'])


# Renaming Columns
renamed_col = {'amount_tsh': 'total_static_head(ft)', 'gps_height': 'height',
               'wpt_name': 'waterpoint_name', 'basin': 'basin_location',
               'lga': 'local_gov_area', 'permit': 'permit_approved',
               'extraction_type_class': 'extraction_method',
               'management_group': 'management_type',
               'payment': 'payment_frequency', 'quality_group': 'quality_of_water',
               'quantity': 'quantity_of_water', 'source': 'water_source'
               }
train_values_df = train_values_df.rename(columns=renamed_col)
test_values_df = test_values_df.rename(columns=renamed_col)


---

## **Sorting Dataframes**

In [20]:
train_values_df = train_values_df.applymap(lambda s:s.lower() if type(s) == str else s)

## Maddie's Filtering

In [34]:
train_values_df['date_recorded'].value_counts()
train_values_df['year_recorded'] = train_values_df['date_recorded'].apply(lambda x: x.year)



In [37]:
train_values_df['year_recorded'].value_counts()

2011    24256
2013    21163
2012     5473
2004       25
2002        1
Name: year_recorded, dtype: int64

In [39]:
train_values_df['well_age'] = train_values_df['year_recorded'] - train_values_df['construction_year'] 

In [52]:
construction_null = train_values_df[train_values_df['construction_year'] == 0]

display_all(construction_null.head(20))

,id,total_static_head(ft),date_recorded,funder,height,installer,longitude,latitude,waterpoint_name,basin_location,subvillage,region,local_gov_area,ward,population,scheme_management,permit_approved,construction_year,extraction_method,management_type,payment_frequency,quality_of_water,quantity_of_water,water_source,source_class,waterpoint_type_group,status_group,year_recorded,well_age
6,19816,0.0,2012-10-01,dwsp,0,dwsp,33.362410,-3.766365e+00,kwa ngomho,internal,ishinabulandi,shinyanga,shinyanga rural,samuye,0,vwc,True,0,handpump,user-group,never pay,good,enough,machine dbh,groundwater,hand pump,non functional,2012,2012
8,53934,0.0,2012-11-03,wateraid,0,water aid,32.711100,-5.146712e+00,kwa ramadhan musa,lake tanganyika,imalauduki,tabora,tabora urban,itetemia,0,vwc,True,0,handpump,user-group,never pay,salty,seasonal,machine dbh,groundwater,hand pump,non functional,2012,2012
12,36957,0.0,2012-10-14,world vision,0,world vision,33.798106,-3.290194e+00,pamba,internal,nkilifa,shinyanga,maswa,busilili,0,wug,False,0,handpump,user-group,other,good,enough,shallow well,groundwater,hand pump,functional,2012,2012
14,53752,0.0,2012-10-20,biore,0,wedeco,34.364073,-3.629333e+00,mwabasabi,internal,mwabasabi,shinyanga,meatu,nkoma,0,wug,True,0,handpump,user-group,never pay,good,enough,shallow well,groundwater,hand pump,functional,2012,2012
21,6091,0.0,2013-02-10,dwsp,0,dwe,0.000000,-2.000000e-08,muungano,lake victoria,ibabachegu,shinyanga,bariadi,ikungulyabashashi,0,wug,False,0,handpump,user-group,unknown,unknown,unknown,shallow well,groundwater,hand pump,functional,2013,2013
30,46804,0.0,2011-03-23,water,0,water,36.108312,-5.791000e+00,ngosi,wami / ruvu,karume,dodoma,chamwino,dabalo,0,vwc,True,0,handpump,user-group,never pay,good,insufficient,shallow well,groundwater,hand pump,functional,2011,2011
36,67359,0.0,2011-07-16,danida,0,central government,33.854345,-9.638520e+00,office ya kijiji,lake nyasa,ndanganyika,mbeya,kyela,katumbasongwe,0,vwc,True,0,gravity,user-group,never pay,good,dry,spring,groundwater,communal standpipe,non functional,2011,2011
37,60048,0.0,2011-03-27,kirde,0,dwe,35.858373,-4.647906e+00,tangi,internal,baura,dodoma,kondoa,mnenia,0,vwc,False,0,gravity,user-group,pay monthly,good,enough,spring,groundwater,communal standpipe,functional,2011,2011
45,12958,0.0,2012-10-22,world vision,0,world vision,33.345260,-3.838998e+00,shimasa,internal,isenegeja,shinyanga,shinyanga rural,samuye,0,wug,True,0,wind-powered,user-group,pay when scheme fails,good,enough,machine dbh,groundwater,communal standpipe,non functional,2012,2012
52,10745,0.0,2011-07-31,kkkt_makwale,0,kkkt _ konde and dwe,33.958163,-9.448122e+00,kwa abeli mwamakula,lake nyasa,isimba,mbeya,kyela,makwale,0,vwc,True,0,gravity,user-group,never pay,good,enough,spring,groundwater,communal standpipe,functional,2011,2011


In [53]:
train_values_df[train_values_df['status_group'] == 'functional needs repair'] = "functional"

In [54]:
train_values_df['status_group'].value_counts()

functional        31467
non functional    19451
Name: status_group, dtype: int64

In [38]:
train_values_df.columns


Index(['id', 'total_static_head(ft)', 'date_recorded', 'funder', 'height',
       'installer', 'longitude', 'latitude', 'waterpoint_name',
       'basin_location', 'subvillage', 'region', 'local_gov_area', 'ward',
       'population', 'scheme_management', 'permit_approved',
       'construction_year', 'extraction_method', 'management_type',
       'payment_frequency', 'quality_of_water', 'quantity_of_water',
       'water_source', 'source_class', 'waterpoint_type_group', 'status_group',
       'year_recorded'],
      dtype='object')

In [22]:
my_df = train_values_df[['extraction_method', 'management_type',
       'payment_frequency', 'quality_of_water', 'quantity_of_water',
       'water_source', 'source_class', 'waterpoint_type_group',
       'status_group']]

In [23]:
#drop payment_frequency
my_df.drop('payment_frequency', axis=1, inplace=True)

#if unknown makes up less than 1%, drop unknown rows from our table 
def drop_unknown(df, column):
    counts = df[column].value_counts(normalize=True)
    if 'unknown' in counts.index and counts['unknown'] < 0.01:
        df.drop(df[df[column] == 'unknown'].index, inplace=True)

#loop through columns to drop unknowns
for column in my_df.columns: 
    drop_unknown(my_df, column)


/var/folders/qh/ns54bt951q74xt4l68nw_m840000gn/T/ipykernel_84899/863275260.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  my_df.drop('payment_frequency', axis=1, inplace=True)


In [26]:
my_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 50234 entries, 0 to 59399
Data columns (total 8 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   extraction_method      50234 non-null  object
 1   management_type        50234 non-null  object
 2   quality_of_water       50234 non-null  object
 3   quantity_of_water      50234 non-null  object
 4   water_source           50234 non-null  object
 5   source_class           50234 non-null  object
 6   waterpoint_type_group  50234 non-null  object
 7   status_group           50234 non-null  object
dtypes: object(8)
memory usage: 3.4+ MB


In [27]:
my_df['source_class'].value_counts(normalize=True)


groundwater    0.769698
surface        0.230302
Name: source_class, dtype: float64

In [51]:
# Function that displays all columns and rows when you use commands like df.head() and df_tail().

def display_all(df):
    with pd.option_context('display.max_rows', 1000):
        with pd.option_context('display.max_columns', 1000):
            display(df)

In [ ]:
display_all(train_values_df.head(100))